#Trace calculation
Compute $\mathrm{Tr}(A+B)$ using expression templates.

In [1]:
%reload_ext pytriqs.magic

In [2]:
%%triqs -u stl
#include <chrono>
#include <iostream>

std::pair<std::vector<int>,array<double,2>> trace_time(int reps){

 using namespace triqs;
 using namespace triqs::arrays;

 std::vector<int> size;
 array<double,2> times(3,100);

 for(int i=0, dim=10; i<100; i++, dim+=10){

  auto A = matrix<double>(dim, dim); A()=0;
  auto B = matrix<double>(dim, dim); B()=0;

  double time1=0, time2=0, time3=0;

  std::chrono::system_clock::time_point start = std::chrono::system_clock::now();
  for(int r=0; r<reps; ++r) auto tr = trace(A+B); // Uses expression template
  std::chrono::duration<double> sec = std::chrono::system_clock::now() - start;
  time1 += sec.count();
 
  start = std::chrono::system_clock::now();
  for(int r=0; r<reps; ++r) auto tr = trace(matrix<double>(A+B)); // Object oriented way
  sec = std::chrono::system_clock::now() - start;
  time2 += sec.count();

  start = std::chrono::system_clock::now();
  double tr;
  for(int r=0; r<reps; ++r){
   tr = 0.;
   for (int ind=0; ind<dim; ind++) // Low level
    tr += A(ind,ind)+B(ind,ind);
  }
  sec = std::chrono::system_clock::now() - start;
  time3 += sec.count();

  std::cerr << tr; // Reuse tr, otherwise optimizer eliminates the loop 

  size.push_back(dim);
  times(0,i)=time1/reps;
  times(1,i)=time2/reps;
  times(2,i)=time3/reps;
 }

 return std::make_pair(size,times);

}


----------  Make error -------
Scanning dependencies of target ext
[100%] Building CXX object CMakeFiles/ext.dir/ext_wrap.cpp.o
In file included from /homepmc/priyanka.seth/SOURCES/TRIQS/install/include/triqs/python_tools/converters/arrays.hpp:2:0,
                 from /mnt/home/priyanka.seth/.cache/ipython/triqs/_triqs_magic_b3a5981bee8165accdbf707f28552a19/ext_wrap.cpp:7:
/homepmc/priyanka.seth/SOURCES/TRIQS/install/include/triqs/python_tools/converters/../wrapper_tools.hpp: In instantiation of 'PyObject* triqs::py_tools::convert_to_python(T&&) [with T = std::pair<std::vector<int, std::allocator<int> >, triqs::arrays::array<double, 2> >; PyObject = _object]':
/mnt/home/priyanka.seth/.cache/ipython/triqs/_triqs_magic_b3a5981bee8165accdbf707f28552a19/ext_wrap.cpp:104:57:   required from here
/homepmc/priyanka.seth/SOURCES/TRIQS/install/include/triqs/python_tools/converters/../wrapper_tools.hpp:51:76: error: incomplete type 'triqs::py_tools::py_converter<std::pair<std::vector<int, std:

In [3]:
s,times = trace_time(1000)

NameError: name 'trace_time' is not defined

In [ ]:
figure(figsize=(15,5))
gs=GridSpec(1,2)
subplot(gs[0])
xlim(-0.0001,500)
ylim(0,0.001)
xlabel('matrix size')
ylabel('t (ms)')
plot(s,times[0],label='TRIQS', lw=4.0)
plot(s,times[1],label='matrix(A+B)', lw=2.0)
plot(s,times[2],label='low level', lw=4.0)
legend()
subplot(gs[1])
xlim(0,500)
ylim(0,0.00001)
xlabel('matrix size')
ylabel('t (ms)')
plot(s,times[0],label='TRIQS', lw=3.0)
plot(s,times[1],label='matrix(A+B)', lw=2.0)
plot(s,times[2],label='low level', lw=3.0)
legend()
savefig('trace.pdf')